# Начало анализа

In [1]:
import pandas as pd
from currency_converter import CurrencyConverter



df = pd.read_csv('_data.csv', index_col=0)
# df.columns
# df['ID  объявления'].value_counts()
# df.head(30)
df.info()
# df['Количество комнат'].value_counts()
# df.loc[:15, 'Площадь, м2']

FileNotFoundError: [Errno 2] No such file or directory: '_data.csv'

### Достаем данные Москвы

In [ ]:
# Город
df['city'] = [row.split(',')[0] if isinstance(row, str) else row for row in df['Адрес']]
df['city'].value_counts()
# df_msk = df[df['Адрес'].str.contains("Москва")].reset_index(drop = True)
df = df[df['city']=='Москва']
df.shape

## Работаем с пропусками

In [ ]:
# Cмотрим NaN значения
df.isna().sum()

### Комнаты

In [ ]:
# Получаем количество комнат
df.loc[:,'Rooms'] = [row.split(',')[0] if isinstance(row, str) else row for row in df['Количество комнат']]
df['Rooms'].unique()
df['Rooms'].value_counts()
df['Rooms'].isna().sum()

# Анализ выборки NaN в графе комнат показал, что это квартиры-студии, заменим NaN на 0
df.loc[df['Rooms'].isna()]['Описание'].head()
df['Rooms'].fillna(0, inplace=True)



### Метро

In [ ]:
# Колонка метро
df['Метро2'] = [row.split(' (')[0] if isinstance(row, str) else row for row in df['Метро']]
df['Метро2'] = [row.split('. ')[1] if isinstance(row, str) else row for row in df['Метро2']]

# df['Метро2'].fillna('Неизвестно', inplace=True)
df['Метро2'].isna().sum()

df['maybe_subway'] = df['Описание'].str.contains('метро', regex=False)
df['maybe_subway']

# нельзя из Описания вытащить новые станции метро 
df.loc[(df['Метро2'].isna()) & (df['maybe_subway']==True)]['Метро2'].count()
df.loc[df['Метро'].isna()]['Дополнительно']
df = df.dropna(subset=['Метро'])

### Адрес

In [ ]:
# Создаем переменную "за мкадом"
df['Адрес'].unique()
df['Not_Moscow'] = df['Адрес'].apply(lambda x: 1 if 'мкад' in str(x).lower() else 0)
df['Not_Moscow'].value_counts()

In [ ]:
# Удаляем концы адреса по типу "ш. Варшавское (4 км до МКАД)"
df['New_address'] = [row.split('ш. ')[0] for row in df['Адрес']]
# df_msk = df_msk[df_msk['Адрес'] != 'Москва, Жилой Поколение кв-л'].reset_index(drop = True)

In [ ]:
# Привязать адрес к районам / спарсить "садовое" из описания

# TBD!


### Площадь

In [ ]:
df['Площадь, м2'].unique()
# Получим общую площадь квартиры
df['total_square'] = [row.split('.')[0] if isinstance(row, str) else row for row in df['Площадь, м2']]
df['total_square'].astype(int)
# df['total_square'].unique()

In [ ]:
# получим площадь кухни
df['Площадь, м2'].unique()
df['kitchen_square'] = [row.split('/')[-1] if isinstance(row, str) else row for row in df['Площадь, м2']]
# df_msk[['Площадь, м2', 'kitchen_square']]
# df_msk['kitchen_square'].isna().sum()

### Животные и дети

In [ ]:
df['Можно с детьми/животными'].unique()
df['ok_pets'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'животными' in str(x).lower() else 0)
df['ok_kids'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'детьми' in str(x).lower() else 0)
# df_msk['ok_kids'].sum()
# df_msk['ok_pets'].sum()

### Балкон

In [ ]:
df['Балкон'].unique()

df['Balconies'] = df['Балкон'].str.extract(r'Балкон \((\d+)\)').astype(float)
df['Loggies'] = df['Балкон'].str.extract(r'Лоджия \((\d+)\)').astype(float)

df['Balconies'].isna().sum()
df['Balconies'] = df['Balconies'].fillna(df['Balconies'].mean())
# df_msk['Число балконов'].isna().sum()
df['Loggies'].isna().sum()
df['Loggies'] = df['Loggies'].fillna(df['Loggies'].mean())

### Санузел

In [ ]:
df['Санузел']
df['Connected_bathroom'] = df['Санузел'].str.extract(r'Совмещенный \((\d+)\)').astype(float)
df['Split_bathroom'] = df['Санузел'].str.extract(r'Раздельный \((\d+)\)').astype(float)

df['Connected_bathroom'] = df['Connected_bathroom'].fillna(df['Connected_bathroom'].mean())
df['Split_bathroom'] = df['Split_bathroom'].fillna(df['Split_bathroom'].mean())
# df_msk['Число совмещенных су'].isna().sum()

### Мебель и кондиционер

In [ ]:
df['Дополнительно']
# Мебель в комнатах
# Кондиционер
df['Furniture'] = df['Дополнительно'].str.extract('(ебель в комнатах)',).astype(str)
df['AC'] = df['Дополнительно'].str.extract('(ондиционер)',).astype(str)

df['Furniture'] = [1 if 'ебель в комнатах' in row else 0 for row in df['Furniture']]
df['AC'] = [1 if 'ондиционер' in row else 0 for row in df['AC']]


### Парковка

In [ ]:
df['Парковка'] = df['Парковка'].replace(to_replace ='открытая', value = 'бесплатная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='наземная', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='многоуровневая', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='подземная', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='на крыше', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].fillna('бесплатная во дворе')
df['Parking'] = df['Парковка']
df = df.drop(['Парковка'], axis = 1)

### Дом (тип, этажность, этаж)

In [ ]:
df['house_type'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
df['floors_amount'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['flat_floor'] = [int(row.split('/')[0]) for row in df['Дом']]
df = df.drop(['Дом'], axis = 1)

### Цена


In [ ]:
c = CurrencyConverter()
def sale(text):
    result = None
    if text.split(' ')[1] == 'руб./':
        result = float(text.split(' ')[0])
    elif text.split(' ')[1] == '$,':
        result = c.convert(float(text.split(' ')[0]), 'USD', 'RUB')
    elif text.split(' ')[1] == '€,':
        result = c.convert(float(text.split(' ')[0]), 'EUR', 'RUB')
    return result
text = df['Цена']
text = text.apply(sale)
df['Price'] = text

In [ ]:
# влючена ли комуналка
df['Utilities'] = df['Цена'].str.contains('Коммунальные услуги включены')
df['Rent_length'] = df['Цена'].str.contains('Срок аренды - Длительный')
df['Prepay'] = df['Цена'].str.contains('Предоплата')

In [ ]:
# лифты

df['Лифт'] = [
    row if row == row else
    ['Пасс (0), Груз (0)' if 5 >= df['floors_amount'].iloc[i]  else
     'Пасс (1), Груз (0)' if 5 <= df['floors_amount'].iloc[i] <= 9  else
     'Пасс (1), Груз (1)' if 10 <= df['floors_amount'].iloc[i] <= 19 else
     'Пасс (2), Груз (1)'][0]
    for i, row in enumerate(df['Лифт'])
]
df['Pass_elevator'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Пасс' else '(0)' for row in df['Лифт']]
df['Cargo_elevator'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Груз' else '(0)' for row in df['Лифт']]

df['Pass_elevator'] = [row.split('(')[1] for row in df['Pass_elevator']]
df['Pass_elevator'] = [int(row.split(')')[0]) for row in df['Pass_elevator']]
df['Cargo_elevator'] = [row.split('(')[1] for row in df['Cargo_elevator']]
df['Cargo_elevator'] = [int(row.split(')')[0]) for row in df['Cargo_elevator']]
df = df.drop(['Лифт'], axis = 1)

### Окна, потолки, мусоропровод, ремонт

In [ ]:
df['windows'] = df["Окна"].fillna('Во двор')

In [ ]:
df['ceiling_height'] = df['Высота потолков, м'].fillna(round(df['Высота потолков, м'].mean(), 2))

In [ ]:
df['Garbage_disposal'] = df['Мусоропровод'].fillna('Нет')

In [ ]:
df['maintenance'] = df['Ремонт'].fillna('Без ремонта')

### Удаление старых / лишних колонок

In [ ]:
df.columns

df.drop(['ID  объявления', 'Площадь, м2', 'Количество комнат', 'Тип', 'Адрес', 'Цена', 'Телефоны', 'Описание',
'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Мусоропровод',
'Ссылка на объявление', 'city', 'maybe_subway'], axis=1, inplace=True)

# df.drop(['ID  объявления'], inplace=True)

In [ ]:
df.dtypes

In [ ]:
df['Метро2']

In [ ]:
df = df.dropna(subset=['Price'])#так как в колонке прайс отсутсвуют некоторые значения мы удаляем объявления без цены

In [ ]:
df.isna().sum()

In [ ]:
df